In [53]:
import pandas as pd
import numpy as np

In [54]:
import pickle

def read_pickle(file):
    f = open(file, 'rb')
    l_d = pickle.load(f)
    return l_d

In [55]:
df_q_navig = pd.read_csv("/content/drive/MyDrive/data_colab/inex_data/query_resource2005_agr.tsv", sep='\t')
print(len(df_q_navig.index))

9


In [56]:
df_rel_navig_2005 = pd.read_csv("/content/drive/MyDrive/data_colab/inex_data/rel_resource2005_agr.tsv", sep='\t')
df_rel_navig_2005

,docid,qid,docno,tag,exhaustivity,specificity,gen_q,strict_q,intent_fin
0,mu/2004/u1012,205,mu/2004/u1012/article[1]/bdy[1]/sec[4]/ss1[3]/...,p,1,1.00,1.00,0,resource
1,mu/2004/u1012,205,mu/2004/u1012/article[1]/bdy[1]/sec[4]/ss1[3],ss,1,0.58,0.58,0,resource
2,mu/2004/u1012,205,mu/2004/u1012/article[1]/bdy[1]/sec[4],sec,1,0.18,0.18,0,resource
3,mu/2004/u1012,205,mu/2004/u1012/article[1]/bdy[1],bdy,1,0.03,0.03,0,resource
4,mu/2004/u1012,205,mu/2004/u1012/article[1],article,1,0.02,0.02,0,resource
...,...,...,...,...,...,...,...,...,...
4941,tp/2001/i0396,283,tp/2001/i0396/article[1]/bdy[1]/sec[6],sec,1,1.00,1.00,0,resource
4942,tp/2001/i0396,283,tp/2001/i0396/article[1]/bdy[1]/sec[5]/ss1[1],ss,2,1.00,2.00,1,resource
4943,tp/2001/i0396,283,tp/2001/i0396/article[1]/bdy[1]/sec[5]/ss1[2],ss,2,1.00,2.00,1,resource
4944,tp/2001/i0396,283,tp/2001/i0396/article[1]/bdy[1]/sec[5]/ss1[3],ss,2,1.00,2.00,1,resource


In [57]:
df_rel_navig_2004 = pd.read_csv("/content/drive/MyDrive/data_colab/inex_data/rel_resource2004_agr.tsv", sep='\t')
df_rel_navig_2004

,docid,qid,docno,tag,exhaustivity,specificity,gen_q,sog_q,strict_q,intent_fin
0,cg/1997/g4075,131,cg/1997/g4075/article[1],article,2,1,0.50,0.10,0,resource
1,cg/1997/g4075,131,cg/1997/g4075/article[1]/fm[1],fm,2,2,0.50,0.50,0,resource
2,cg/1997/g4075,131,cg/1997/g4075/article[1]/fm[1]/abs[1],abs,2,2,0.50,0.50,0,resource
3,cg/1997/g4075,131,cg/1997/g4075/article[1]/fm[1]/abs[1]/p[1],p,2,2,0.50,0.50,0,resource
4,cg/1999/g5020,131,cg/1999/g5020/article[1],article,1,1,0.25,0.10,0,resource
...,...,...,...,...,...,...,...,...,...,...
7308,tp/2002/i0059,155,tp/2002/i0059/article[1]/bm[1]/bib[1]/bibl[1],bibl,1,1,0.25,0.10,0,resource
7309,tp/2002/i0059,155,tp/2002/i0059/article[1]/bm[1]/bib[1]/bibl[1]/...,bb,1,2,0.25,0.25,0,resource
7310,tp/2002/i0059,155,tp/2002/i0059/article[1]/bm[1]/bib[1]/bibl[1]/...,atl,1,2,0.25,0.25,0,resource
7311,tp/2002/i0059,155,tp/2002/i0059/article[1]/bm[1]/bib[1]/bibl[1]/...,bb,1,3,0.50,0.75,0,resource


In [58]:
df_rel_navig_2004 = df_rel_navig_2004[['qid','docno','sog_q']]
df_rel_navig_2004 = df_rel_navig_2004.rename(columns={'sog_q': 'label'})
df_rel_navig_2005 = df_rel_navig_2005.rename(columns={'gen_q': 'label'})
df_rel_navig_2005["qid"] = df_rel_navig_2005["qid"].astype(str)
df_rel_navig_2004["qid"] = df_rel_navig_2004["qid"].astype(str)

In [59]:
import numpy as np

def dcg_at_k(relevance_scores, k):
    """Calculate DCG for the given list of relevance scores up to position k."""
    relevance_scores = np.asfarray(relevance_scores)[:k]
    if relevance_scores.size:
        return np.sum(relevance_scores / np.log2(np.arange(2, relevance_scores.size + 2)))
    return 0

def ndcg_at_k(predicted_scores, ideal_scores, k):
    """Calculate NDCG for the given list of predicted and ideal relevance scores up to position k."""
    idcg = dcg_at_k(sorted(ideal_scores, reverse=True), k)
    if idcg == 0:
        return 0
    dcg = dcg_at_k(predicted_scores, k)
    return dcg / idcg

In [60]:
def ndcg_apply(df,k):
  grouped_result = df.groupby('qid')["label"].agg(list)
  l_ndcg = []
  for predicted_scores in grouped_result.to_list():
    ideal_scores = sorted(predicted_scores, reverse=True)
    ndcg_score = ndcg_at_k(predicted_scores, ideal_scores,k)
    #print(f"NDCG at 10: {ndcg_score:.4f}")
    l_ndcg.append(ndcg_score)
  return l_ndcg

BM25

In [133]:
bm25_res = pd.read_csv('/content/drive/MyDrive/data_colab/inex_data/res_with_features_navig_2004_features_and_sections.tsv', sep='\t')
bm25_res["qid"] = bm25_res["qid"].astype(str)

In [135]:
bm25_res = bm25_res.merge(df_rel_navig_2004[['qid', 'docno', 'label']], on=['qid', 'docno'], how='left')
bm25_res['label'] = bm25_res['label'].fillna(0)
bm25_res.head()

,qid,docid,docno,rank,score_bm25,query,intent_fin,score_tfidf,text,sum_tfidf,...,mean_tfidf,var_tfidf,document_length,jaccard_similarity,cosine_similarity,jaccard_similarity_se,cosine_similarity_se,query_in_text,exact_match,label
0,131,20978,co/1999/r8020/article[1]/bdy[1]/sec[4],0,43.128174,jiawei han data mining,resource,24.993204,"CONSTRAINT-BASED, MULTIDIMENSIONAL DATA MINING...",7938.707777,...,5.478749,4.533846,102,0.019417,0.252217,0.000000,0.000000,0,0,0.5
1,131,33673,tk/1995/k0847/article[1]/bdy[1]/sec[6],1,40.990351,jiawei han data mining,resource,23.643479,"SPECIAL SECTIONS 1996 1996, plan special secti...",7938.707777,...,5.478749,4.533846,77,0.013699,0.063372,0.000000,0.000000,0,0,0.0
2,131,33898,tk/2001/k0393/article[1]/bdy[1]/sec[1],2,39.806578,jiawei han data mining,resource,23.102250,great pleasure introduce five new members Edit...,7938.707777,...,5.478749,4.533846,97,0.020619,0.117851,0.020619,0.117851,0,0,0.0
3,131,20985,co/1999/r8034/article[1]/bdy[1]/sec[4],3,38.534675,jiawei han data mining,resource,22.433131,ISSUE five articles issue cover gamut topics i...,7938.707777,...,5.478749,4.533846,211,0.009390,0.146944,0.000000,0.000000,0,0,0.1
4,131,36378,tk/1996/k0193/article[1]/bdy[1]/sec[1],4,37.371258,jiawei han data mining,resource,21.390704,INTRODUCTION pleasure introduce four distingui...,7938.707777,...,5.478749,4.533846,131,0.008197,0.044677,0.008197,0.044677,0,0,0.0


In [124]:
mean_bm25 = np.mean(ndcg_apply(bm25_res,5))
print("NDCG@5 BM25", mean_bm25)

NDCG@5 BM25 0.24883304673408105


1. INTENT-AGNOSTIC

In [108]:
train_data = pd.read_csv('/content/drive/MyDrive/data_colab/inex_data/res_with_features_navig_2005_features_and_sections.tsv', sep='\t')
train_data["qid"] = train_data["qid"].astype(str)

In [110]:
train_data = train_data.merge(df_rel_navig_2005[['qid', 'docno', 'label']], on=['qid', 'docno'], how='left')
train_data['label'] = train_data['label'].fillna(0)
train_data.head()

,qid,docid,docno,rank,score_bm25,query,intent_fin,score_tfidf,text,sum_tfidf,...,mean_tfidf,var_tfidf,document_length,jaccard_similarity,cosine_similarity,jaccard_similarity_se,cosine_similarity_se,query_in_text,exact_match,label
0,205,30819,co/1996/r8091/article[1]/bdy[1]/sec[1],0,30.697313,marshall mcluhan,resource,16.782770,"pervasive medium age television, radio, even p...",1279.258178,...,5.076421,8.202943,97,0.010204,0.098058,0.010204,0.098058,0,0,0.25
1,205,5940,mu/1996/u2003/article[1]/bdy[1]/sec[1],1,30.196907,marshall mcluhan,resource,16.509190,Marshall McLuhan imagined world future global ...,1279.258178,...,5.076421,8.202943,102,0.010000,0.098533,0.010000,0.098533,1,1,1.00
2,205,2480,pd/1998/p1013/article[1]/bdy[1]/sec[6],2,29.664973,marshall mcluhan,resource,16.218372,"SEEDS TAKING ROOT Although 250,000 individuals...",1279.258178,...,5.076421,8.202943,120,0.008197,0.090536,0.000000,0.000000,0,0,0.00
3,205,10321,mi/1996/m6054/article[1]/bdy[1]/sec[7],3,28.567031,marshall mcluhan,resource,15.618107,Community Probably microprocessor's greatest s...,1279.258178,...,5.076421,8.202943,134,0.007463,0.085436,0.000000,0.000000,0,0,0.58
4,205,44457,cs/1999/c6001/article[1]/bdy[1]/sec[1],4,28.463019,marshall mcluhan,resource,15.563131,Marshall McLuhan observed technology extension...,1279.258178,...,5.076421,8.202943,263,0.003759,0.060302,0.003759,0.060302,1,1,0.41


In [112]:
test_data = pd.read_csv('/content/drive/MyDrive/data_colab/inex_data/res_with_features_navig_2004_features_and_sections.tsv',sep = '\t')
test_data
test_data["qid"] = test_data["qid"].astype(str)

In [114]:
test_data = test_data.merge(df_rel_navig_2004[['qid', 'docno', 'label']], on=['qid', 'docno'], how='left')
test_data['label'] = test_data['label'].fillna(0)
test_data.head()

,qid,docid,docno,rank,score_bm25,query,intent_fin,score_tfidf,text,sum_tfidf,...,mean_tfidf,var_tfidf,document_length,jaccard_similarity,cosine_similarity,jaccard_similarity_se,cosine_similarity_se,query_in_text,exact_match,label
0,131,20978,co/1999/r8020/article[1]/bdy[1]/sec[4],0,43.128174,jiawei han data mining,resource,24.993204,"CONSTRAINT-BASED, MULTIDIMENSIONAL DATA MINING...",7938.707777,...,5.478749,4.533846,102,0.019417,0.252217,0.000000,0.000000,0,0,0.5
1,131,33673,tk/1995/k0847/article[1]/bdy[1]/sec[6],1,40.990351,jiawei han data mining,resource,23.643479,"SPECIAL SECTIONS 1996 1996, plan special secti...",7938.707777,...,5.478749,4.533846,77,0.013699,0.063372,0.000000,0.000000,0,0,0.0
2,131,33898,tk/2001/k0393/article[1]/bdy[1]/sec[1],2,39.806578,jiawei han data mining,resource,23.102250,great pleasure introduce five new members Edit...,7938.707777,...,5.478749,4.533846,97,0.020619,0.117851,0.020619,0.117851,0,0,0.0
3,131,20985,co/1999/r8034/article[1]/bdy[1]/sec[4],3,38.534675,jiawei han data mining,resource,22.433131,ISSUE five articles issue cover gamut topics i...,7938.707777,...,5.478749,4.533846,211,0.009390,0.146944,0.000000,0.000000,0,0,0.1
4,131,36378,tk/1996/k0193/article[1]/bdy[1]/sec[1],4,37.371258,jiawei han data mining,resource,21.390704,INTRODUCTION pleasure introduce four distingui...,7938.707777,...,5.478749,4.533846,131,0.008197,0.044677,0.008197,0.044677,0,0,0.0


In [68]:
relevance_labels = np.array(train_data['label'].to_list())

# Define the number of bins for label mapping (0 to 6 corresponds to 7 gains)
n_bins = 7

# Use numpy to digitize the labels into bins
int_labels = np.digitize(relevance_labels, np.linspace(min(relevance_labels), max(relevance_labels), n_bins))
print(int_labels[10:])

# Adjust to ensure labels are within 0 to 6
int_labels = int_labels - 1 # Digitize returns 1-indexed bins, so subtract 1

label_gain = sorted(set(int_labels))
print(label_gain)

[1 2 1 ... 1 1 1]
[0, 1, 2, 3, 4, 5, 6]


In [69]:
relevance_labels_test = test_data['label'].to_list(
)
# Remove duplicates by converting the list to a set, then convert back to a sorted list
label_gain_test = sorted(set(relevance_labels_test))
print(label_gain_test)

xint_labels_test = [label_gain_test.index(label) for label in relevance_labels_test]

[0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]


In [70]:
import numpy as np

def dcg_at_k(relevance_scores, k):
    """Calculate DCG for the given list of relevance scores up to position k."""
    relevance_scores = np.asfarray(relevance_scores)[:k]
    if relevance_scores.size:
        return np.sum(relevance_scores / np.log2(np.arange(2, relevance_scores.size + 2)))
    return 0

def ndcg_at_k(predicted_scores, ideal_scores, k):
    """Calculate NDCG for the given list of predicted and ideal relevance scores up to position k."""
    idcg = dcg_at_k(sorted(ideal_scores, reverse=True), k)
    if idcg == 0:
        return 0
    dcg = dcg_at_k(predicted_scores, k)
    return dcg / idcg


In [71]:
def ndcg_apply(df,k):
  grouped_result = df.groupby('qid')["label"].agg(list)
  l_ndcg = []
  for predicted_scores in grouped_result.to_list():
    ideal_scores = sorted(predicted_scores, reverse=True)
    ndcg_score = ndcg_at_k(predicted_scores, ideal_scores,k)
    #print(f"NDCG at 10: {ndcg_score:.4f}")
    l_ndcg.append(ndcg_score)
  return l_ndcg

In [72]:
!pip install pandas lightgbm

In [73]:
import lightgbm as lgb

In [74]:
#Prepare the features (X) and labels (y)

X_train_na = train_data[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text']].values  # Including the user intent feature
y_train_na = train_data['label'].values  # Use the scaled labels
print(len(X_train_na))

# Extract the group sizes (number of documents per query) for training
group_train_na = train_data.groupby('qid').size().to_numpy()

# Create the LightGBM dataset
train_data_na = lgb.Dataset(X_train_na, label=int_labels, group=group_train_na)
print(len(X_train_na))
print(len(relevance_labels))

10311
10311
10311


In [75]:
# Prepare the features (X) and labels (y) for test data
X_test_na = test_data[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text']].values
y_test_na = test_data['label'].values  # Use the actual labels

# Extract the group sizes (number of documents per query) for test data
group_test_na = test_data.groupby('qid').size().to_numpy()

# Create LightGBM dataset for test set
test_data_na = lgb.Dataset(X_test_na, label=int_labels_test, group=group_test_na)


In [76]:
#min_data_in_leaf previousely 50, num leaves 16 max depth 6

params_lm = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'boosting': 'gbdt',
    'num_leaves': 16,
    'max_depth': 6,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'min_data_in_leaf': 100,  # Increase this to regularize more
    'label_gain': label_gain,  # Increase this to regularize more
    'lambda_l1': 0.1,  # Add L1 regularization
    'lambda_l2': 0.1,  # Add L2 regularization
    'verbose': -1
}



In [77]:
import matplotlib.pyplot as plt


#Initialize dictionaries to store results
evals_result = {'train': [], 'test': []}

# Custom callback to track evaluation results manually and print every 10 rounds
def record_evaluation(env):
    results = env.evaluation_result_list
    train_ndcg = results[0][2]  # Training NDCG (assuming first is train)
    test_ndcg = results[1][2]   # Test NDCG (assuming second is test)

    # Append to the evals_result dict
    evals_result['train'].append(train_ndcg)
    evals_result['test'].append(test_ndcg)

    # Print every 10 rounds
    if (env.iteration + 1) % 10 == 0:
        print(f"Round {env.iteration + 1}: Train NDCG = {train_ndcg}, Test NDCG = {test_ndcg}")

# Train the model and track NDCG on training and test sets
ranker_lm_na = lgb.train(
    params_lm,
    train_data_na,
    num_boost_round=150,
    valid_sets=[train_data_na, test_data_na],  # Include both training and test sets for evaluation
    valid_names=['train', 'test'],  # Give names to the datasets
    callbacks=[record_evaluation, lgb.early_stopping(stopping_rounds=50)]
)


Training until validation scores don't improve for 50 rounds
Round 10: Train NDCG = 0.6222222222222222, Test NDCG = 0.5868704456388283
Round 20: Train NDCG = 0.6222222222222222, Test NDCG = 0.5940343865135421
Round 30: Train NDCG = 0.6222222222222222, Test NDCG = 0.5940343865135421
Round 40: Train NDCG = 0.6222222222222222, Test NDCG = 0.5940343865135421
Round 50: Train NDCG = 0.6222222222222222, Test NDCG = 0.5940343865135421
Early stopping, best iteration is:
[1]	train's ndcg@1: 0.314815	train's ndcg@2: 0.471465	train's ndcg@3: 0.440423	train's ndcg@4: 0.420682	train's ndcg@5: 0.418203	test's ndcg@1: 0.65	test's ndcg@2: 0.594638	test's ndcg@3: 0.526337	test's ndcg@4: 0.527762	test's ndcg@5: 0.526475


In [78]:
def apply_lambdamart_na(df, model):
    ranked_results = []
    queries = df['qid'].unique()
    for qid in queries:
        subset = df[df['qid'] == qid]
        features = subset[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text']].values
        scores = model.predict(features)
        subset = subset.assign(lambdamart_score=scores)
        ranked_results.append(subset.sort_values(by='lambdamart_score', ascending=False))

    return pd.concat(ranked_results)


In [79]:
result_lm_na = apply_lambdamart_na(test_data, ranker_lm_na)

In [80]:
result_lm_na = result_lm_na.reset_index(drop=True)
result_lm_na["qid"] = result_lm_na["qid"].astype(str)

2.INTENT-AWARE

In [81]:
# Prepare the features (X) and labels (y)
X_train_ia = train_data[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text','jaccard_similarity','cosine_similarity','jaccard_similarity_se','cosine_similarity_se','exact_match']].values  # Including the user intent feature
y_train_ia = train_data['label'].values  # Use the scaled labels
print(len(X_train_ia))

# Extract the group sizes (number of documents per query) for training
group_train_ia = train_data.groupby('qid').size().to_numpy()

# Create the LightGBM dataset
train_data_ia = lgb.Dataset(X_train_ia, label=int_labels, group=group_train_ia)
print(len(X_train_ia))
print(len(relevance_labels))

10311
10311
10311


In [82]:
# Prepare the features (X) and labels (y) for test data
X_test_ia = test_data[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text','jaccard_similarity','cosine_similarity','jaccard_similarity_se','cosine_similarity_se','exact_match']].values
y_test_ia = test_data['label'].values  # Use the actual labels

# Extract the group sizes (number of documents per query) for test data
group_test_ia = test_data.groupby('qid').size().to_numpy()

# Create LightGBM dataset for test set
test_data_ia = lgb.Dataset(X_test_ia, label=int_labels_test, group=group_test_ia)
print(len(X_test_ia))
print(len(relevance_labels_test))

6901
6901


In [83]:
#Initialize dictionaries to store results
evals_result = {'train': [], 'test': []}

# Custom callback to track evaluation results manually and print every 10 rounds
def record_evaluation(env):
    results = env.evaluation_result_list
    train_ndcg = results[0][2]  # Training NDCG (assuming first is train)
    test_ndcg = results[1][2]   # Test NDCG (assuming second is test)

    # Append to the evals_result dict
    evals_result['train'].append(train_ndcg)
    evals_result['test'].append(test_ndcg)

    # Print every 10 rounds
    if (env.iteration + 1) % 10 == 0:
        print(f"Round {env.iteration + 1}: Train NDCG = {train_ndcg}, Test NDCG = {test_ndcg}")


# Train the model with early stopping using callbacks
ranker_lm_ia = lgb.train(
    params_lm,
    train_data_ia,
    num_boost_round=150,  # Set maximum number of boosting rounds
    valid_sets=[train_data_ia, test_data_ia],  # Include both training and test sets for evaluation
    valid_names=['train', 'test'],
    callbacks=[record_evaluation, lgb.early_stopping(stopping_rounds=15)]  # Early stopping after 10 rounds without improvement
)



Training until validation scores don't improve for 15 rounds
Round 10: Train NDCG = 0.7740740740740741, Test NDCG = 0.8025660852344223
Early stopping, best iteration is:
[3]	train's ndcg@1: 0.596296	train's ndcg@2: 0.617044	train's ndcg@3: 0.621772	train's ndcg@4: 0.617465	train's ndcg@5: 0.60256	test's ndcg@1: 0.683333	test's ndcg@2: 0.660729	test's ndcg@3: 0.620859	test's ndcg@4: 0.573977	test's ndcg@5: 0.595255


In [84]:
def apply_lambdamart_ia(df, model):
    ranked_results = []
    queries = df['qid'].unique()
    for qid in queries:
        subset = df[df['qid'] == qid]
        features = subset[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text','jaccard_similarity','cosine_similarity','jaccard_similarity_se','cosine_similarity_se','exact_match']].values
        scores = model.predict(features)
        subset = subset.assign(lambdamart_score=scores)
        ranked_results.append(subset.sort_values(by='lambdamart_score', ascending=False))

    return pd.concat(ranked_results)

In [85]:
result_lm_ia = apply_lambdamart_ia(test_data, ranker_lm_ia)
result_lm_ia = result_lm_ia.reset_index(drop=True)
result_lm_ia["qid"] = result_lm_ia["qid"].astype(str)

In [127]:
mean_bm25 = round(np.mean(ndcg_apply(bm25_res,5)),4)
print("NDCG@5 BM25:", mean_bm25)
mean_lm_na = round(np.mean(ndcg_apply(result_lm_na,5)),4)
print("NDCG@5 lambdamart non intent aware:", mean_lm_na)
mean_lm_ia = round(np.mean(ndcg_apply(result_lm_ia,5)),4)
print("NDCG@5 lambdamart intent aware:", mean_lm_ia)


NDCG@5 BM25: 0.2488
NDCG@5 lambdamart non intent aware: 0.3307
NDCG@5 lambdamart intent aware: 0.328


In [128]:
improv = (np.mean(ndcg_apply(result_lm_na,5)) - np.mean(ndcg_apply(bm25_res,5)))/np.mean(ndcg_apply(bm25_res,5))*100
print(f"improvement na is {round(improv,2)} percent")

improvement na is 32.91 percent


In [129]:
improv = (np.mean(ndcg_apply(result_lm_ia,5)) - np.mean(ndcg_apply(bm25_res,5)))/np.mean(ndcg_apply(bm25_res,5))*100
print(f"improvement ia is {round(improv,2)} percent")

improvement ia is 31.83 percent


In [89]:
#Prepare the features (X) and labels (y)

X_train_sr_na = train_data[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text']].values  # Including the user intent feature
y_train_sr_na = train_data['label'].values  # Use the scaled labels
print(len(X_train_sr_na))

# Extract the group sizes (number of documents per query) for training
group_train_sr_na = train_data.groupby('qid').size().to_numpy()

# Create the LightGBM dataset
train_data_sr_na = lgb.Dataset(X_train_sr_na, label=int_labels, group=group_train_sr_na)
print(len(X_train_sr_na))
print(len(relevance_labels))

10311
10311
10311


In [90]:
# Prepare the features (X) and labels (y) for test data
X_test_sr_na = test_data[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text']].values
y_test_sr_na = test_data['label'].values  # Use the actual labels

# Extract the group sizes (number of documents per query) for test data
group_test_sr_na = test_data.groupby('qid').size().to_numpy()

# Create LightGBM dataset for test set
test_data_sr_na = lgb.Dataset(X_test_sr_na, label=int_labels_test, group=group_test_sr_na)


In [91]:
# Define the parameters for Soft rank
params_sr = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'label_gain': label_gain,  # Adjust this to cover the range of your scaled labels
    'verbose': -1
}

In [92]:
import matplotlib.pyplot as plt


#Initialize dictionaries to store results
evals_result = {'train': [], 'test': []}

# Custom callback to track evaluation results manually and print every 10 rounds
def record_evaluation(env):
    results = env.evaluation_result_list
    train_ndcg = results[0][2]  # Training NDCG (assuming first is train)
    test_ndcg = results[1][2]   # Test NDCG (assuming second is test)

    # Append to the evals_result dict
    evals_result['train'].append(train_ndcg)
    evals_result['test'].append(test_ndcg)

    # Print every 10 rounds
    if (env.iteration + 1) % 10 == 0:
        print(f"Round {env.iteration + 1}: Train NDCG = {train_ndcg}, Test NDCG = {test_ndcg}")

# Train the model and track NDCG on training and test sets
ranker_sr_na = lgb.train(
    params_sr,
    train_data_sr_na,
    num_boost_round=150,
    valid_sets=[train_data_sr_na, test_data_sr_na],  # Include both training and test sets for evaluation
    valid_names=['train', 'test'],  # Give names to the datasets
    callbacks=[record_evaluation, lgb.early_stopping(stopping_rounds=20)]
)



Training until validation scores don't improve for 20 rounds
Round 10: Train NDCG = 0.7407407407407407, Test NDCG = 0.7146055296113217
Round 20: Train NDCG = 0.7703703703703704, Test NDCG = 0.7605380320940518
Early stopping, best iteration is:
[1]	train's ndcg@1: 0.37037	train's ndcg@2: 0.40179	train's ndcg@3: 0.433945	train's ndcg@4: 0.424992	train's ndcg@5: 0.430503	test's ndcg@1: 0.65	test's ndcg@2: 0.555953	test's ndcg@3: 0.520193	test's ndcg@4: 0.522182	test's ndcg@5: 0.526503


In [93]:
def apply_soft_rank_na(df):
  qid_list = df['qid'].unique()
  ranked_results = []
  for qid in qid_list:
      subset = df[df['qid'] == qid]
      features = subset[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text']].values  # Only numeric features for prediction
      scores = ranker_sr_na.predict(features)
      subset = subset.assign(soft_rank_score=scores)
      ranked_results.append(subset.sort_values(by='soft_rank_score', ascending=False))
  return pd.concat(ranked_results)

# Apply the AdaRank model to re-rank the documents
result_sr_na = apply_soft_rank_na(test_data)

# Reset index to get a clean DataFrame
result_sr_na = result_sr_na.reset_index(drop=True)
result_sr_na["qid"] = result_sr_na["qid"].astype(str)

In [94]:
# Prepare the features (X) and labels (y)
X_train_sr_ia = train_data[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text','jaccard_similarity','cosine_similarity','jaccard_similarity_se','cosine_similarity_se','exact_match']].values  # Including the user intent feature
y_train_sr_ia = train_data['label'].values  # Use the scaled labels
print(len(X_train_sr_ia))

# Extract the group sizes (number of documents per query) for training
group_train_sr_ia = train_data.groupby('qid').size().to_numpy()

# Create the LightGBM dataset
train_data_sr_ia = lgb.Dataset(X_train_sr_ia, label=int_labels, group=group_train_sr_ia)
print(len(X_train_sr_ia))
print(len(relevance_labels))

10311
10311
10311


In [95]:
# Prepare the features (X) and labels (y) for test data
X_test_sr_ia = test_data[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text','jaccard_similarity','cosine_similarity','jaccard_similarity_se','cosine_similarity_se','exact_match']].values
y_test_sr_ia = test_data['label'].values  # Use the actual labels

# Extract the group sizes (number of documents per query) for test data
group_test_sr_ia = test_data.groupby('qid').size().to_numpy()

# Create LightGBM dataset for test set
test_data_sr_ia = lgb.Dataset(X_test_sr_ia, label=int_labels_test, group=group_test_sr_ia)
print(len(X_test_sr_ia))
print(len(relevance_labels_test))

6901
6901


In [96]:
#Initialize dictionaries to store results
evals_result = {'train': [], 'test': []}

# Custom callback to track evaluation results manually and print every 10 rounds
def record_evaluation(env):
    results = env.evaluation_result_list
    train_ndcg = results[0][2]  # Training NDCG (assuming first is train)
    test_ndcg = results[1][2]   # Test NDCG (assuming second is test)

    # Append to the evals_result dict
    evals_result['train'].append(train_ndcg)
    evals_result['test'].append(test_ndcg)

    # Print every 10 rounds
    if (env.iteration + 1) % 10 == 0:
        print(f"Round {env.iteration + 1}: Train NDCG = {train_ndcg}, Test NDCG = {test_ndcg}")


# Train the model with early stopping using callbacks
ranker_sr_ia = lgb.train(
    params_sr,
    train_data_sr_ia,
    num_boost_round=150,  # Set maximum number of boosting rounds
    valid_sets=[train_data_sr_ia, test_data_sr_ia],  # Include both training and test sets for evaluation
    valid_names=['train', 'test'],
    callbacks=[record_evaluation, lgb.early_stopping(stopping_rounds=15)]  # Early stopping after 10 rounds without improvement
)


Training until validation scores don't improve for 15 rounds
Round 10: Train NDCG = 0.7962962962962963, Test NDCG = 0.8438362596588878
Early stopping, best iteration is:
[2]	train's ndcg@1: 0.685185	train's ndcg@2: 0.702028	train's ndcg@3: 0.733867	train's ndcg@4: 0.707807	train's ndcg@5: 0.692396	test's ndcg@1: 0.583333	test's ndcg@2: 0.690569	test's ndcg@3: 0.667438	test's ndcg@4: 0.617672	test's ndcg@5: 0.638999


In [97]:
def apply_soft_rank_ia(df):
  qid_list = df['qid'].unique()
  ranked_results = []
  for qid in qid_list:
      subset = df[df['qid'] == qid]
      features = subset[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text','jaccard_similarity','cosine_similarity','jaccard_similarity_se','cosine_similarity_se','exact_match']].values  # Only numeric features for prediction
      scores = ranker_sr_ia.predict(features)
      subset = subset.assign(soft_rank_score=scores)
      ranked_results.append(subset.sort_values(by='soft_rank_score', ascending=False))
  return pd.concat(ranked_results)

In [98]:
result_sr_ia = apply_soft_rank_ia(test_data)
result_sr_ia = result_sr_ia.reset_index(drop=True)
result_sr_ia["qid"] = result_sr_ia["qid"].astype(str)

In [130]:
mean_bm25 = round(np.mean(ndcg_apply(bm25_res,5)),4)
print("NDCG@5 BM25:", mean_bm25)
mean_sr_na = round(np.mean(ndcg_apply(result_sr_na,5)),4)
print("NDCG@5 soft rank non intent aware:", mean_sr_na)
mean_sr_ia = round(np.mean(ndcg_apply(result_sr_ia,5)),4)
print("NDCG@5 soft rank intent aware:", mean_sr_ia)

NDCG@5 BM25: 0.2488
NDCG@5 soft rank non intent aware: 0.3479
NDCG@5 soft rank intent aware: 0.3829


In [131]:
improv = (np.mean(ndcg_apply(result_sr_na,5)) - np.mean(ndcg_apply(bm25_res,5)))/np.mean(ndcg_apply(bm25_res,5))*100
print(f"improvement na is {round(improv,2)} percent")

improvement na is 39.83 percent


In [132]:
improv = (np.mean(ndcg_apply(result_sr_ia,5)) - np.mean(ndcg_apply(bm25_res,5)))/np.mean(ndcg_apply(bm25_res,5))*100
print(f"improvement na is {round(improv,2)} percent")

improvement na is 53.89 percent


In [102]:
from scipy.stats import ttest_rel
nb_el = 5

baseline = ndcg_apply(bm25_res,nb_el)
ndcg_lm_na = ndcg_apply(result_lm_na,nb_el)
ndcg_lm_ia = ndcg_apply(result_lm_ia,nb_el)
ndcg_sr_na = ndcg_apply(result_sr_na,nb_el)
ndcg_sr_ia = ndcg_apply(result_sr_ia,nb_el)




# Perform t-tests
t_stat1, p_val1 = ttest_rel(baseline, ndcg_lm_na)
t_stat2, p_val2 = ttest_rel(baseline, ndcg_lm_ia)
t_stat3, p_val3 = ttest_rel(baseline, ndcg_sr_na)
t_stat4, p_val4 = ttest_rel(baseline, ndcg_sr_ia)

# Collect p-values
p_values = [p_val1, p_val2,p_val3,p_val4]
print(p_values)

[0.0836718190317838, 0.3390520636586988, 0.19544807886279914, 0.22658299119649428]


In [103]:
# Number of tests
num_tests = len(p_values)
print(num_tests)

# Apply Bonferroni correction
bonferroni_corrected_pvals = [min(p * num_tests, 1.0) for p in p_values]
print(bonferroni_corrected_pvals)

4
[0.3346872761271352, 1.0, 0.7817923154511965, 0.9063319647859771]


In [104]:
alpha = 0.05

# Apply Bonferroni correction
bonferroni_corrected_pvals = [min(p * num_tests, 1.0) for p in p_values]
l_names = ['Lambdamart na','Lambdamart_ia','Softrank na','Softrank_ia']

# Check for significance and print results
for i, (t_stat, p_val, corrected_pval) in enumerate(zip([t_stat1, t_stat2,t_stat3,t_stat4], p_values, bonferroni_corrected_pvals)):
    is_significant = corrected_pval <= alpha
    significance_str = "Significant" if is_significant else "Not Significant"
    print(f"Result {i+1}:")
    print(l_names[i])
    print(f"  t-statistic = {t_stat:.4f}")
    print(f"  original p-value = {p_val:.4f}")
    print(f"  Bonferroni corrected p-value = {corrected_pval:.4f} ({significance_str})")
    print()

Result 1:
Lambdamart na
  t-statistic = -2.2920
  original p-value = 0.0837
  Bonferroni corrected p-value = 0.3347 (Not Significant)

Result 2:
Lambdamart_ia
  t-statistic = -1.0847
  original p-value = 0.3391
  Bonferroni corrected p-value = 1.0000 (Not Significant)

Result 3:
Softrank na
  t-statistic = -1.5527
  original p-value = 0.1954
  Bonferroni corrected p-value = 0.7818 (Not Significant)

Result 4:
Softrank_ia
  t-statistic = -1.4276
  original p-value = 0.2266
  Bonferroni corrected p-value = 0.9063 (Not Significant)

